# Generate automated session reports for all participants

In [1]:
# import packages
import pandas as pd
import numpy as np
import mne
import mne_bids
from matplotlib import pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
import plotly.offline as py
import plotly.io as pio
pio.renderers.default = "plotly_mimetype+notebook"
import q1k_init_tools as qit
import glob
import warnings
from nbconvert import HTMLExporter
from nbconvert.preprocessors import ExecutePreprocessor
from nbformat import read
import os
import papermill as pm
warnings.filterwarnings('ignore')

### Select task parameters and file paths

In [6]:
# Select kind of data experimental or control group

dataset_group = "experimental"

#if dataset_group == "control":
#    # Control group data
#    project_path = "/home/james/q1k/pilot/q1k-external-pilot/"
#    task_id_in = "ap"
#    task_id_in_et = "ap"
#    task_id_out = "ap"
#    #subject_id = '002'
#    session_id = '01'
#    run_id = '1'

#elif dataset_group == "experimental":
# Experimental group data

project_path = "/project/def-emayada/q1k/experimental/HSJ/"
init_path = "code/q1k_eeget_init/"
task_id_in = "PLR"
task_id_in_et = "PLR"
task_id_out = "PLR"
#subject_id = 'Q1K_HSJ_100123_F1'
run_id = '1'
session_id = '01'
site_code = 'HSJ' #'MHC' or 'HSJ'
sourcedata_path = "sourcedata/" 
html_reports_path = "session_reports/" + task_id_in  + '/'
et_sync = True


In [7]:
# Sanity check to see which task you would like to make reports for 
print(task_id_in)
et_sync

PLR


True

In [8]:
# Generate list of session reports already processed so they are not processed again in next step
processed_sessions = []

for subject in glob.glob(project_path + init_path + "session_reports/" + task_id_in + "/Q1k*.html"): 
    processed_sessions.append(subject.split('\\')[1][:-5])

print('Existing session reports:')
processed_sessions

Existing session reports:


[]

## Generate html session reports for all participants

In [9]:
glob.glob(project_path + sourcedata_path + "eeg/Q1K*/Q1K*" + task_id_in + '_*.mff')

['/project/def-emayada/q1k/experimental/HSJ/sourcedata/eeg/Q1K_HSJ_1525-1026_P/Q1K_HSJ_1525-1026_P_PLR_20240613_031556.mff',
 '/project/def-emayada/q1k/experimental/HSJ/sourcedata/eeg/Q1K_HSJ_1525-1026_M1/Q1K_HSJ_1525-1026_M1_PLR_20240613_121650.mff',
 '/project/def-emayada/q1k/experimental/HSJ/sourcedata/eeg/Q1K_HSJ_1025-1042_M1/Q1K_HSJ_1025-1042_M1_PLR_20240715_124201.mff',
 '/project/def-emayada/q1k/experimental/HSJ/sourcedata/eeg/Q1K_HSJ_1025-1061_M/Q1K_HSJ_1025-1061_M1_PLR_20240718_031746.mff',
 '/project/def-emayada/q1k/experimental/HSJ/sourcedata/eeg/Q1K_HSJ_1525-1057_M1/Q1K_HSJ_1525-1057_M1_PLR_20240722_025056.mff',
 '/project/def-emayada/q1k/experimental/HSJ/sourcedata/eeg/Q1K_HSJ_1525_1009_S1/Q1K_HSJ_1525_1009_S1_PLR_20240806_115800.mff',
 '/project/def-emayada/q1k/experimental/HSJ/sourcedata/eeg/Q1K_HSJ_100162_M1/Q1K_HSJ_100162_M1_PLR_20240530_020053.mff',
 '/project/def-emayada/q1k/experimental/HSJ/sourcedata/eeg/Q1K_HSJ_1525-1045_S1/Q1K_HSJ_1525-1045_S1_PLR_20240805_113905

In [ ]:
# Make sure the output directory exists
if not os.path.exists(html_reports_path):
    os.makedirs(html_reports_path)

# Create a list of sessions with errors
error_subjects = []

#Loop through existing sessions and execute the q1k_generate_individual_reports.ipynb if a session report does not already exist
for file in glob.glob(project_path + sourcedata_path + "eeg/Q1K*/Q1K*" + task_id_in + '_*.mff'):
    print('Current data file: ' + file)
    # Select anything after the Q1K and before the AEP
    #subject_id = file.split('_')[2]
    site_part = file.split(site_code + "_")
    subject_number = site_part[1].split("_")[0]
    print('Participant number: ' + subject_number)
    subject_relation = site_part[1].split("_")[1].split("/")[0]
    print('Participant relation: ' + subject_relation)

    subject_id_in = subject_number + "_" + subject_relation
    print('Participant ID input: ' + subject_id_in)
    subject_id_out = subject_number.replace('_','').replace('-','') + subject_relation
    print('Participant ID output: ' + subject_id_out)
    
    # Skip sessions that have a session report in the output directory
    print(subject_id_in)
    if subject_id_in in processed_sessions:
        print(subject_id_in + ' has already been processed')
        continue    

    # Define paths
    input_notebook = project_path + init_path +'q1k_generate_individual_reports.ipynb'
    print('Input notebook: ' + input_notebook)

    # Make sure the directory exists
    if not os.path.exists(f'{project_path}{init_path}session_reports/{task_id_in}/executed_notebooks/'):
        os.makedirs(f'{project_path}{init_path}session_reports/{task_id_in}/executed_notebooks/')
    
    output_notebook = f'{project_path}{init_path}{html_reports_path}executed_notebooks/{subject_id_in}_{task_id_in}_executed.ipynb'
    print('Output notebook file: ' + output_notebook)
    output_html = f'{project_path}{init_path}{html_reports_path}{subject_id_in}_{task_id_in}.html'
    print('Output HTML file: ' + output_html)

    try:
        # Execute the notebook
        pm.execute_notebook(input_notebook, output_notebook, kernel_name = 'q1k_env', parameters=dict(subject_id_in=subject_id_in, subject_id_out=subject_id_out,task_id_in=task_id_in, task_id_in_et=task_id_in_et,
         task_id_out=task_id_out, run_id=run_id, session_id=session_id, project_path=project_path, dataset_group=dataset_group, site_code=site_code))

        # Convert executed notebook to HTML
        html_exporter = HTMLExporter()
        html_exporter.exclude_input = True

        (body, resources) = html_exporter.from_filename(output_notebook)

        # Save HTML output
        with open(output_html, 'w', encoding='utf-8') as f:
            f.write(body)

        print(f"HTML report saved for {subject_id_in}.")
    

    except Exception as e:
        # Handle the error 
        error_subjects.append(subject_id_in)
        print(f"Error while processing {subject_id_in}: {e}")

# Print out the list of subjects with errors
print( "These subjects have errors: " + str(error_subjects) + " and need to be reprocessed")

Current data file: /project/def-emayada/q1k/experimental/HSJ/sourcedata/eeg/Q1K_HSJ_1525-1026_P/Q1K_HSJ_1525-1026_P_PLR_20240613_031556.mff
Participant number: 1525-1026
Participant relation: P
Participant ID input: 1525-1026_P
Participant ID output: 15251026P
1525-1026_P
Input notebook: /project/def-emayada/q1k/experimental/HSJ/code/q1k_eeget_init/q1k_generate_individual_reports.ipynb
Output notebook file: /project/def-emayada/q1k/experimental/HSJ/code/q1k_eeget_init/session_reports/PLR/executed_notebooks/1525-1026_P_PLR_executed.ipynb
Output HTML file: /project/def-emayada/q1k/experimental/HSJ/code/q1k_eeget_init/session_reports/PLR/1525-1026_P_PLR.html


Executing:   0%|          | 0/51 [00:00<?, ?cell/s]

Error while processing 1525-1026_P: 
---------------------------------------------------------------------------
Exception encountered at "In [14]":
---------------------------------------------------------------------------
ValueError                                Traceback (most recent call last)
Cell In[14], line 9
      7 din_diffs, din_diffs_time = qit.get_din_diff(eeg_events, eeg_event_dict, din_str)
      8 #build the figure...
----> 9 fig=px.scatter(x=din_diffs_time, y=din_diffs)
     10 fig.update_layout(title='Time between EEG DIN events of interest')
     11 fig.update_xaxes(title_text='Time of event(ms)')

File /project/def-emayada/q1k/experimental/q1k_env/lib/python3.11/site-packages/plotly/express/_chart_types.py:66, in scatter(data_frame, x, y, color, symbol, size, hover_name, hover_data, custom_data, text, facet_row, facet_col, facet_col_wrap, facet_row_spacing, facet_col_spacing, error_x, error_x_minus, error_y, error_y_minus, animation_frame, animation_group, categor

Executing:   0%|          | 0/51 [00:00<?, ?cell/s]

### Visualize all reports at once

In [ ]:
#for file in glob.glob("./../../sourcefiles/EEG/Q1K*/Q1K" + "*AEP" + '*.mff'):
#    # Select anything after the Q1K and before the AEP
#    subject_id = file.split('\\')[1]
#    # Skip sessions that have already been processed
#    print(subject_id)
#    if subject_id in processed_sessions:
#        print (subject_id + ' has already been processed')
#        continue    

#    # Handle participants with error 
#    try:
#        %run -i ./q1k_generate_individual_reports.ipynb --subject_id={subject_id}
#    except Exception as e:
#    # Handle the error (e.g., log it, skip participant, etc.)
#        print(f"Error while reading raw data for {subject_id}: {e}")
